# Análisis del riesgo de incumplimiento de los prestatarios

Tu proyecto consiste en preparar un informe para la división de préstamos de un banco. Deberás averiguar si el estado civil y el número de hijos de un cliente tienen un impacto en el incumplimiento de pago de un préstamo. El banco ya tiene algunos datos sobre la solvencia crediticia de los clientes.

Tu informe se tendrá en cuenta al crear una **puntuación de crédito** para un cliente potencial. La **puntuación de crédito** se utiliza para evaluar la capacidad de un prestatario potencial para pagar su préstamo.


## Abre el archivo de datos y mira la información general. 


In [1]:
import pandas as pd
import numpy as np


In [2]:
data = pd.read_csv('credit_scoring_eng.csv')
data


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house
1,1,-4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase
2,0,-5623.422610,33,Secondary Education,1,married,0,M,employee,0,23341.752,purchase of the house
3,3,-4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding
...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,-4529.316663,43,secondary education,1,civil partnership,1,F,business,0,35966.698,housing transactions
21521,0,343937.404131,67,secondary education,1,married,0,F,retiree,0,24959.969,purchase of a car
21522,1,-2113.346888,38,secondary education,1,civil partnership,1,M,employee,1,14347.610,property
21523,3,-3112.481705,38,secondary education,1,married,0,M,employee,1,39054.888,buying my own car


## Ejercicio 1. Exploración de datos

**Descripción de los datos**
- `children` - el número de hijos en la familia
- `days_employed` - experiencia laboral en días
- `dob_years` - la edad del cliente en años
- `education` - la educación del cliente
- `education_id` - identificador de educación
- `family_status` - estado civil
- `family_status_id` - identificador de estado civil
- `gender` - género del cliente
- `income_type` - tipo de empleo
- `debt` - ¿había alguna deuda en el pago de un préstamo?
- `total_income` - ingreso mensual
- `purpose` - el propósito de obtener un préstamo

[Ahora vamos a explorar nuestros datos. Querrás ver cuántas columnas y filas hay, observa algunas filas para identificar posibles problemas con los datos.]

In [3]:
print(data['total_income'].dtype)

float64


In [4]:
# Vamos a ver cuántas filas y columnas tiene nuestro conjunto de datos
data.shape


(21525, 12)

In [5]:
data.dtypes

children              int64
days_employed       float64
dob_years             int64
education            object
education_id          int64
family_status        object
family_status_id      int64
gender               object
income_type          object
debt                  int64
total_income        float64
purpose              object
dtype: object

In [6]:
data.columns

Index(['children', 'days_employed', 'dob_years', 'education', 'education_id',
       'family_status', 'family_status_id', 'gender', 'income_type', 'debt',
       'total_income', 'purpose'],
      dtype='object')

In [7]:
# vamos a mostrar las primeras filas N
data.head()


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house
1,1,-4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase
2,0,-5623.422610,33,Secondary Education,1,married,0,M,employee,0,23341.752,purchase of the house
3,3,-4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding


En total la base de datos tiene 12 columnas y 21525 filas. se destribuyen en formato float, int y object. 
a simple vista se puede detectar que en las columnas "days_employed"  y "total_income " tienen valores ausentes.

In [8]:
# Obtener información sobre los datos
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [9]:
(data.isna()
 .sum()
 .sort_values(ascending=False)
 .reset_index()
 .rename(columns = {'index' : 'Variables', 0: 'Missing'})
).T

,0,1,2,3,4,5,6,7,8,9,10,11
Variables,days_employed,total_income,children,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,purpose
Missing,2174,2174,0,0,0,0,0,0,0,0,0,0


De la base de datos existen dos cateogiras con valores ausentes: "days_employed"  y "total_income "

In [10]:
# Veamos la tabla filtrada con valores ausentes de la primera columna donde faltan datos

data_filtrada=data[data['days_employed'].isnull()]
data_filtrada

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,secondary education,1,civil partnership,1,M,retiree,0,NaN,to have a wedding
26,0,NaN,41,secondary education,1,married,0,M,civil servant,0,NaN,education
29,0,NaN,63,secondary education,1,unmarried,4,F,retiree,0,NaN,building a real estate
41,0,NaN,50,secondary education,1,married,0,F,civil servant,0,NaN,second-hand car purchase
55,0,NaN,54,secondary education,1,civil partnership,1,F,retiree,1,NaN,to have a wedding
...,...,...,...,...,...,...,...,...,...,...,...,...
21489,2,NaN,47,Secondary Education,1,married,0,M,business,0,NaN,purchase of a car
21495,1,NaN,50,secondary education,1,civil partnership,1,F,employee,0,NaN,wedding ceremony
21497,0,NaN,48,BACHELOR'S DEGREE,0,married,0,F,business,0,NaN,building a property
21502,1,NaN,42,secondary education,1,married,0,F,employee,0,NaN,building a real estate


In [11]:
data_filtrada['total_income'].count()

#al contar la columa en nuestra nueva variable no figuran datos, es decir todos estan vacios 

0

A simple vista  parecen simetrico ua que tienen el mismo numero e valores ausentes entre ambas 

In [12]:
# Apliquemos múltiples condiciones para filtrar datos y veamos el número de filas en la tabla filtrada.

data_filtrada=data[(data['days_employed'].isnull())&(data['total_income'].isnull())]
print(f'tenemos un total de {len(data_filtrada)} valores ausentes ')

tenemos un total de 2174 valores ausentes 


In [13]:
porcentaje_ausentes=round(len(data_filtrada)/len(data)* 100,1)
print(f' El {(porcentaje_ausentes)} % son valores ausentes ')

 El 10.1 % son valores ausentes 


**Conclusión intermedia**

El número de filas en la tabla filtrada coincide con el número de valores ausentes, de un total de  21525, 2174 son valores ausentes. los datos ausentes parecen simetricos, ya que los valores ausentes corresponden a los mismos usuario entre ambas columnas. se desconoce el motivo real de esto, puede ser debido a que los uuarios quisieron omitir esos campos o pudo haber sido una caida del sistema o carga.

En total el 10,1% son valores ausentes y la magnitud de la proporcion lo definira el cliente, ese porcentaje puede ser tanto significativo como minimo para él. en caso que sea valiosa se debera completar los valores ausentes de acuerdo a su criterio. Pero en este caso, despues de haber notificado el problema, se tienen que identificar los valores ausentes, y de acuerdo al motivo, se rellenaran para no generar un problema al analisis final. 


In [14]:
data_limpia= data.dropna() #eliminacion de los datos nulos
data_limpia

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house
1,1,-4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase
2,0,-5623.422610,33,Secondary Education,1,married,0,M,employee,0,23341.752,purchase of the house
3,3,-4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding
...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,-4529.316663,43,secondary education,1,civil partnership,1,F,business,0,35966.698,housing transactions
21521,0,343937.404131,67,secondary education,1,married,0,F,retiree,0,24959.969,purchase of a car
21522,1,-2113.346888,38,secondary education,1,civil partnership,1,M,employee,1,14347.610,property
21523,3,-3112.481705,38,secondary education,1,married,0,M,employee,1,39054.888,buying my own car


In [15]:
data_limpia.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19351 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          19351 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         19351 non-null  int64  
 3   education         19351 non-null  object 
 4   education_id      19351 non-null  int64  
 5   family_status     19351 non-null  object 
 6   family_status_id  19351 non-null  int64  
 7   gender            19351 non-null  object 
 8   income_type       19351 non-null  object 
 9   debt              19351 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           19351 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 1.9+ MB


In [16]:
# Vamos a investigar a los clientes que no tienen datos sobre la característica identificada y la columna con los valores ausentes


variable = "income_type"

serie_data= data_limpia[variable]
serie_filtrada= data_filtrada[variable]

print(serie_data.value_counts())
print("--------------------------------")
print(serie_filtrada.value_counts())

employee                       10014
business                        4577
retiree                         3443
civil servant                   1312
unemployed                         2
student                            1
entrepreneur                       1
paternity / maternity leave        1
Name: income_type, dtype: int64
--------------------------------
employee         1105
business          508
retiree           413
civil servant     147
entrepreneur        1
Name: income_type, dtype: int64


In [17]:
# Comprobación de la distribución
n_serie_data=len(serie_data)
n_serie_filtrada=len(serie_filtrada)

print('Distribucion de serie completa:')
print(round(serie_data.value_counts()/ n_serie_data * 100,1))
print('-----------------------')
print('Distribucion de serie valores ausentes')
print(round(serie_filtrada.value_counts()/ n_serie_filtrada * 100,1))


Distribucion de serie completa:
employee                       51.7
business                       23.7
retiree                        17.8
civil servant                   6.8
unemployed                      0.0
student                         0.0
entrepreneur                    0.0
paternity / maternity leave     0.0
Name: income_type, dtype: float64
-----------------------
Distribucion de serie valores ausentes
employee         50.8
business         23.4
retiree          19.0
civil servant     6.8
entrepreneur      0.0
Name: income_type, dtype: float64


No hay correlacion en los datos, los valores ausentes son al azar


**Conclusión intermedia**

Hasta ahora no se puede idntificar ningun patro, el porcentaje de numero ausentes no tiene una proporcion significativa entre  las distintas variables y la serie completa


In [18]:
# Comprueba otras razones y patrones que podrían llevar a valores ausentes
print('Informacion de datos limpios;')
print(data.describe())
print('-------------------------')
print('Informacion de datos ausentes;')
print(data_filtrada.describe())



Informacion de datos limpios;
           children  days_employed     dob_years  education_id  \
count  21525.000000   19351.000000  21525.000000  21525.000000   
mean       0.538908   63046.497661     43.293380      0.817236   
std        1.381587  140827.311974     12.574584      0.548138   
min       -1.000000  -18388.949901      0.000000      0.000000   
25%        0.000000   -2747.423625     33.000000      1.000000   
50%        0.000000   -1203.369529     42.000000      1.000000   
75%        1.000000    -291.095954     53.000000      1.000000   
max       20.000000  401755.400475     75.000000      4.000000   

       family_status_id          debt   total_income  
count      21525.000000  21525.000000   19351.000000  
mean           0.972544      0.080883   26787.568355  
std            1.420324      0.272661   16475.450632  
min            0.000000      0.000000    3306.762000  
25%            0.000000      0.000000   16488.504500  
50%            0.000000      0.000000   23202

**Conclusión intermedia**

Con esto se puede detectar que los valores ausentes de total_income y days_employe no tienen una relacion con las otras variables, no hay un patron que determine el comportamiento de esto

In [19]:
# Comprobación de otros patrones: explica cuáles
variable = "education_id"

serie_data= data_limpia[variable]
serie_filtrada= data_filtrada[variable]

print(serie_data.value_counts())
print("--------------------------------")
print(serie_filtrada.value_counts())

1    13693
0     4716
2      675
3      261
4        6
Name: education_id, dtype: int64
--------------------------------
1    1540
0     544
2      69
3      21
Name: education_id, dtype: int64


**Conclusiones**

Las columnas days_employe y total_income presentar un porcentaje minimo de valores ausentes, el 1%, esto no genera un numero significativa alto. 
Al intentar detectar el motivo de esto o la deteccion de un patron entre las otras variables, no se logro identificar un patron en ello, por lo que se puede determinar que son aleatorios. y ninguna otra variable determina esto. 
al tener valores ausentes, primero que nada se debe dar aviso al superior o a los encargados de los datos, para luego poder aplicar las medidas necesarias, ya que se pueden eliminar, restaurar o reemplazar los valores

## Ejercicio 2 Transformación de datos


#### 1.2 Education 

In [20]:
# Veamos todos los valores en la columna de educación para verificar si será necesario corregir la ortografía y qué habrá que corregir exactamente
variable= 'education'
print(data[variable].unique())


["bachelor's degree" 'secondary education' 'Secondary Education'
 'SECONDARY EDUCATION' "BACHELOR'S DEGREE" 'some college'
 'primary education' "Bachelor's Degree" 'SOME COLLEGE' 'Some College'
 'PRIMARY EDUCATION' 'Primary Education' 'Graduate Degree'
 'GRADUATE DEGREE' 'graduate degree']


In [21]:
# Arregla los registros si es necesario, convertir lo mayuscula a minuscula
data['education_lower'] = data['education'].str.lower()


In [22]:
# Comprobar todos los valores en la columna para asegurarnos de que los hayamos corregido
variable= 'education_lower'

print(data[variable].unique())

["bachelor's degree" 'secondary education' 'some college'
 'primary education' 'graduate degree']


#### 1.2 Children

In [23]:
# Veamos la distribución de los valores en la columna `children`
variable= 'children'
print(data[variable].unique())

[ 1  0  3  2 -1  4 20  5]


Hay numeros negativos (-1)  Y 20 ,y en esta caso no se puede tener menos un hijo, para solucionar esto se reemplazara por 1 positivo Y 20 por 2 . ya que pudo haber sido un error de tipeo o sistema


In [24]:

print(data.groupby('children')['education_id'].count())

children
-1        47
 0     14149
 1      4818
 2      2055
 3       330
 4        41
 5         9
 20       76
Name: education_id, dtype: int64


In [25]:
# [arregla los datos según tu decisión]
# rememplazar 20 por 2 y -1 por 1

data["children"]= data["children"].replace({20:2,-1:1})




In [26]:
# Comprobar la columna `children` de nuevo para asegurarnos de que todo está arreglado

print(data["children"].unique())

print("------------------------------------")
print(data.groupby('children')['dob_years'].count())


[1 0 3 2 4 5]
------------------------------------
children
0    14149
1     4865
2     2131
3      330
4       41
5        9
Name: dob_years, dtype: int64


#### 1.3 days_employed 

In [27]:
print(data['days_employed'].unique())

[-8437.67302776 -4024.80375385 -5623.42261023 ... -2113.3468877
 -3112.4817052  -1984.50758853]


In [28]:
# Encuentra datos problemáticos en `days_employed`, si existen, y calcula el porcentaje
datos_negativos= len(data[data['days_employed'] < 0])
datos_total=len(data['days_employed'])
print(datos_negativos/datos_total *100,1)

73.89547038327527 1


El porcentaje es alto de valores negativos

In [29]:
# Aborda los valores problemáticos, si existen.
data['days_employed']= data['days_employed'].abs()

In [30]:
# Comprueba el resultado - asegúrate de que esté arreglado
datos_negativos= len(data[data['days_employed'] < 0])
print(datos_negativos)

0


In [31]:
data['days_employed'].describe()

count     19351.000000
mean      66914.728907
std      139030.880527
min          24.141633
25%         927.009265
50%        2194.220567
75%        5537.882441
max      401755.400475
Name: days_employed, dtype: float64

In [32]:
#Reemplazar los valores por el tercerr quintil

data.loc[data['days_employed']>5537, 'days_employed'] =5537 

In [33]:
data['days_employed'].describe()

count    19351.000000
mean      2733.737672
std       2000.879779
min         24.141633
25%        927.009265
50%       2194.220567
75%       5536.900576
max       5537.000000
Name: days_employed, dtype: float64

Se reemplazaron los valores negativos por positivos

#### 1.4 Columna Dob_years

In [34]:
# Revisa `dob_years` en busca de valores sospechosos y cuenta el porcentaje
variable= 'dob_years'
print("valores unicos:")
print(data[variable].unique())
print("----------------------------")
print("dsitribucion de los datos:")
data_cero=len(data[data["dob_years"] ==0])
data_nocero=len(data[data["dob_years"] !=0])
print(f' El porcentaje es de { data_cero/data_nocero *100.1} % ')

valores unicos:
[42 36 33 32 53 27 43 50 35 41 40 65 54 56 26 48 24 21 57 67 28 63 62 47
 34 68 25 31 30 20 49 37 45 61 64 44 52 46 23 38 39 51  0 59 29 60 55 58
 71 22 73 66 69 19 72 70 74 75]
----------------------------
dsitribucion de los datos:
 El porcentaje es de 0.4719053398058252 % 


Al tener solo el 0,4 % de edad igual a cero hace que sea poco significativo, por ende, en esta este caso se reemplazaran los valores por la media 

In [35]:
# Resuelve los problemas en la columna `dob_years`, si existen
data_nocero=data[data["dob_years"] !=0]
dato_nocero_med= data_nocero.mean()
print(dato_nocero_med)


children                0.479649
days_employed        2734.055351
dob_years              43.497479
education_id            0.817914
family_status_id        0.971294
debt                    0.080891
total_income        26794.435857
dtype: float64


C:\Users\clara\AppData\Local\Temp\ipykernel_28448\3498231075.py:3: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  dato_nocero_med= data_nocero.mean()


In [36]:
# Resuelve los problemas en la columna `dob_years`, si existen
data["dob_years"]=data["dob_years"].replace({0:43})
print(data)

       children  days_employed  dob_years            education  education_id  \
0             1    5537.000000         42    bachelor's degree             0   
1             1    4024.803754         36  secondary education             1   
2             0    5537.000000         33  Secondary Education             1   
3             3    4124.747207         32  secondary education             1   
4             0    5537.000000         53  secondary education             1   
...         ...            ...        ...                  ...           ...   
21520         1    4529.316663         43  secondary education             1   
21521         0    5537.000000         67  secondary education             1   
21522         1    2113.346888         38  secondary education             1   
21523         3    3112.481705         38  secondary education             1   
21524         2    1984.507589         40  secondary education             1   

           family_status  family_status

In [37]:
# Comprueba el resultado - asegúrate de que esté arreglado

print(len(data[data["dob_years"] == 0]))

0


#### 1.5 Family status

In [38]:
data.groupby('family_status')['dob_years'].count()

print("----------------------------")
print("informacion")
print(data.family_status.info())


----------------------------
informacion
<class 'pandas.core.series.Series'>
RangeIndex: 21525 entries, 0 to 21524
Series name: family_status
Non-Null Count  Dtype 
--------------  ----- 
21525 non-null  object
dtypes: object(1)
memory usage: 168.3+ KB
None


#### 1.6 Gender

In [39]:
# Veamos los valores en la columna

# Veamos los valores de la columna
variable= 'gender'
print("valores unicos:")
print(data[variable].unique())

print("----------------------------")
print("ditribucion de los datos:")
print(data.groupby(variable)['dob_years'].count())

print("----------------------------")
print("informacion")
print(data[variable].info())

valores unicos:
['F' 'M' 'XNA']
----------------------------
ditribucion de los datos:
gender
F      14236
M       7288
XNA        1
Name: dob_years, dtype: int64
----------------------------
informacion
<class 'pandas.core.series.Series'>
RangeIndex: 21525 entries, 0 to 21524
Series name: gender
Non-Null Count  Dtype 
--------------  ----- 
21525 non-null  object
dtypes: object(1)
memory usage: 168.3+ KB
None


In [40]:
# Aborda los valores problemáticos, si existen
data[data['gender'] == "XNA"]
data["gender"]=data["gender"].replace({"XNA":"F"})

In [41]:
# Comprueba el resultado - asegúrate de que esté arreglado

data["gender"]=data["gender"].replace({"XNA":"F"})
print(data["gender"].unique())

['F' 'M']


#### 1.6  income_type

In [42]:
# Veamos los valores en la columna

variable= 'income_type'
print("valores unicos:")
print(data[variable].unique())

print("----------------------------")
print("ditribucion de los datos:")
print(data.groupby(variable)['dob_years'].count())

print("----------------------------")
print("informacion")
print(data[variable].info())

valores unicos:
['employee' 'retiree' 'business' 'civil servant' 'unemployed'
 'entrepreneur' 'student' 'paternity / maternity leave']
----------------------------
ditribucion de los datos:
income_type
business                        5085
civil servant                   1459
employee                       11119
entrepreneur                       2
paternity / maternity leave        1
retiree                         3856
student                            1
unemployed                         2
Name: dob_years, dtype: int64
----------------------------
informacion
<class 'pandas.core.series.Series'>
RangeIndex: 21525 entries, 0 to 21524
Series name: income_type
Non-Null Count  Dtype 
--------------  ----- 
21525 non-null  object
dtypes: object(1)
memory usage: 168.3+ KB
None


In [43]:
# Aborda los valores problemáticos, si existen
data['income_type'] = np.where(data['income_type']=='entrepreneur','business', data['income_type'])

# Excluir los valores que no representan un valor significativo 
data['income_type']=np.where(data['income_type']=='unemployed', np.nan, data['income_type'])
data['income_type']=np.where(data['income_type']=='student', np.nan, data['income_type'])
data['income_type']=np.where(data['income_type']=='paternity / maternity leave', np.nan, data['income_type'])
data = data.dropna(subset=['income_type'])

In [44]:
# Comprueba el resultado - asegúrate de que esté arreglado
variable= 'income_type'
print("valores unicos:")
print(data[variable].unique())

print("----------------------------")
print("ditribucion de los datos:")
print(data.groupby(variable)['dob_years'].count())

print("----------------------------")
print("informacion")
print(data[variable].info())


valores unicos:
['employee' 'retiree' 'business' 'civil servant']
----------------------------
ditribucion de los datos:
income_type
business          5087
civil servant     1459
employee         11119
retiree           3856
Name: dob_years, dtype: int64
----------------------------
informacion
<class 'pandas.core.series.Series'>
Int64Index: 21521 entries, 0 to 21524
Series name: income_type
Non-Null Count  Dtype 
--------------  ----- 
21521 non-null  object
dtypes: object(1)
memory usage: 336.3+ KB
None


#### 1.7  Revision de duplicados 


In [45]:
# Comprobar los duplicados

data.duplicated().sum()

54

In [46]:
# Aborda los duplicados, si existen
data=data.drop_duplicates().reset_index(drop=True)  #Hay que guardarla en otra variable

In [47]:
# Última comprobación para ver si tenemos duplicados
data.duplicated().sum()

0

In [48]:
# Comprueba el tamaño del conjunto de datos que tienes ahora, después de haber ejecutado estas primeras manipulaciones

data.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21467.000000,19347.000000,21467.000000,21467.000000,21467.000000,21467.000000,19347.000000
mean,0.480179,2733.530169,43.483067,0.817254,0.973634,0.081008,26789.677035
std,0.755868,2000.819531,12.217154,0.548500,1.421034,0.272854,16475.929295
min,0.000000,24.141633,19.000000,0.000000,0.000000,0.000000,3306.762000
25%,0.000000,927.009265,34.000000,1.000000,0.000000,0.000000,16494.016000
50%,0.000000,2194.216968,43.000000,1.000000,0.000000,0.000000,23203.786000
75%,1.000000,5535.883055,53.000000,1.000000,1.000000,0.000000,32551.225000
max,5.000000,5537.000000,75.000000,4.000000,4.000000,1.000000,362496.645000



Se eliminaron 54 duplicados, que representan el 0.25% del total. al eliminarlos quedo un total de 21471 datos 


# Ejercicio 3 Trabajar con valores ausentes

Al tener una simplicacion de datos analizare las columnas education_id y family_status_id

In [49]:
#Education_id


data.pivot=data.pivot_table(index=["education_id","education_lower",],
                           values="children"
                           )
print(data.pivot)



                                  children
education_id education_lower              
0            bachelor's degree    0.524481
1            secondary education  0.465363
2            some college         0.467742
3            primary education    0.482270
4            graduate degree      0.666667


In [50]:
#Family status_id


data.pivot=data.pivot_table(index=["family_status_id","family_status",],
                           values="children"
                           )
print(data.pivot)



                                    children
family_status_id family_status              
0                married            0.572274
1                civil partnership  0.461557
2                widow / widower    0.157456
3                divorced           0.434310
4                unmarried          0.232823


In [51]:
# Encuentra los diccionarios

data.set_index('education_id')['family_status_id'].to_dict()

{0: 1, 1: 0, 2: 4, 3: 1, 4: 3}

### Restaurar valores ausentes en `total_income`

Los valores ausentes se encuentran en las variables days_employed y total_income. hay que reemplazar los valores ausentes con el criterio que mas se adecue a ellos

Para detectar los valores ausentes es necesario crear una caterogia de la edad de los clientes, para detectar el total 

Adultos: clientes de 0 -24 anos.

Seniors: clientes de 25-64 anos.

Retirados: clientes mayores de 65 anos.



In [52]:
print(data['total_income'].dtype)

float64


In [53]:
# Vamos a escribir una función que calcule la categoría de edad
def age_group(dob_years):
    if dob_years <= 24:
        return 'adulto joven'
    elif (dob_years >= 25) and (dob_years < 65):
        return 'adulto mayor'
    else:
        return 'retirado'
    
    
data["age_group"] = data['dob_years'].apply(age_group)




In [54]:
# Prueba si la función funciona bien
data.pivot=data.pivot_table(index=['age_group',"dob_years"],
                           values="children"
                           )
print(data.pivot)


                        children
age_group    dob_years          
adulto joven 19         0.000000
             20         0.235294
             21         0.189189
             22         0.247253
             23         0.324111
             24         0.382576
adulto mayor 25         0.408964
             26         0.607843
             27         0.640974
             28         0.669980
             29         0.724265
             30         0.817844
             31         0.903226
             32         0.886051
             33         0.912220
             34         0.900166
             35         0.887987
             36         0.911552
             37         1.000000
             38         0.852596
             39         0.774081
             40         0.728171
             41         0.632013
             42         0.676174
             43         0.548124
             44         0.506422
             45         0.457661
             46         0.424947
          

In [55]:
print(age_group(10))
print(age_group(17))
print(age_group(60))
print(age_group(73))

adulto joven
adulto joven
adulto mayor
retirado


In [56]:
# Crea una tabla sin valores ausentes y muestra algunas de sus filas para asegurarte de que se ve bien
data_not_nan = data[data['total_income'].isnull()==False]
data_not_nan.head()


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,education_lower,age_group
0,1,5537.000000,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house,bachelor's degree,adulto mayor
1,1,4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase,secondary education,adulto mayor
2,0,5537.000000,33,Secondary Education,1,married,0,M,employee,0,23341.752,purchase of the house,secondary education,adulto mayor
3,3,4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education,secondary education,adulto mayor
4,0,5537.000000,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding,secondary education,adulto mayor


In [57]:
print(data['total_income'].dtype)

float64


In [58]:
display(data_not_nan.corr())

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
children,1.000000,-0.180776,-0.318074,-0.025805,-0.159767,0.025512,0.031455
days_employed,-0.180776,1.000000,0.570387,0.038294,-0.058406,-0.083757,-0.057476
dob_years,-0.318074,0.570387,1.000000,0.062315,-0.069160,-0.072963,-0.056063
education_id,-0.025805,0.038294,0.062315,1.000000,0.010696,0.052582,-0.178908
family_status_id,-0.159767,-0.058406,-0.069160,0.010696,1.000000,0.022610,-0.009151
debt,0.025512,-0.083757,-0.072963,0.052582,0.022610,1.000000,-0.012115
total_income,0.031455,-0.057476,-0.056063,-0.178908,-0.009151,-0.012115,1.000000


In [59]:
data_not_nan.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
education_lower     0
age_group           0
dtype: int64

In [60]:
# Examina los valores medios de los ingresos en función de los factores que identificaste
print(data_not_nan.groupby(['education_id','education_lower'])['total_income'].mean())
print('*'*50)
print(data_not_nan.groupby(['education_id','education_lower'])['total_income'].median())

education_id  education_lower    
0             bachelor's degree      33146.650067
1             secondary education    24596.766073
2             some college           29045.443644
3             primary education      21144.882211
4             graduate degree        27960.024667
Name: total_income, dtype: float64
**************************************************
education_id  education_lower    
0             bachelor's degree      28054.5310
1             secondary education    21837.9890
2             some college           25618.4640
3             primary education      18741.9760
4             graduate degree        25161.5835
Name: total_income, dtype: float64


In [61]:
print(data['total_income'].dtype)

float64


In [62]:
print(data_not_nan.groupby(["family_status_id","family_status"])['total_income'].mean())
print('*'*50)
print(data_not_nan.groupby(["family_status_id","family_status"])['total_income'].median())

family_status_id  family_status    
0                 married              27045.005027
1                 civil partnership    26692.891057
2                 widow / widower      22984.208556
3                 divorced             27189.354550
4                 unmarried            26938.515847
Name: total_income, dtype: float64
**************************************************
family_status_id  family_status    
0                 married              23390.0570
1                 civil partnership    23185.4770
2                 widow / widower      20514.1900
3                 divorced             23515.0960
4                 unmarried            23149.1755
Name: total_income, dtype: float64


In [63]:

print(data_not_nan.groupby("age_group")['total_income'].mean())
print(data_not_nan.groupby("age_group")['total_income'].median())

age_group
adulto joven    22712.337081
adulto mayor    27210.173808
retirado        21542.650450
Name: total_income, dtype: float64
age_group
adulto joven    20574.8955
adulto mayor    23540.3150
retirado        18471.3910
Name: total_income, dtype: float64


In [64]:
variable="gender"
print(data_not_nan.groupby(variable)['total_income'].mean())
print(data_not_nan.groupby(variable)['total_income'].median())

gender
F    24656.877768
M    30912.679377
Name: total_income, dtype: float64
gender
F    21465.165
M    26840.997
Name: total_income, dtype: float64


In [65]:
print(data_not_nan.groupby("income_type")['total_income'].mean())
print(data_not_nan.groupby("income_type")['total_income'].median())

income_type
business         32397.165026
civil servant    27343.729582
employee         25820.841683
retiree          21940.394503
Name: total_income, dtype: float64
income_type
business         27583.3600
civil servant    24071.6695
employee         22815.1035
retiree          18962.3180
Name: total_income, dtype: float64


Las variables 'gender','education_id'e 'income_type' son las que más se adaptan a la identificacion del calculo de los ingresos

In [66]:
#  Escribe una función que usaremos para completar los valores ausentes
data_not_nan_avg=data_not_nan.groupby(['gender','education_id','income_type'])['total_income'].transform("median")
data_not_nan_avg

0        24590.6965
1        19860.3340
2        25005.8240
3        25005.8240
4        18046.5560
            ...    
21462    23181.0850
21463    18046.5560
21464    25005.8240
21465    25005.8240
21466    19860.3340
Name: total_income, Length: 19347, dtype: float64

In [67]:
print(data['total_income'].dtype)

float64


In [68]:
# Comprueba si funciona
data_not_nan_avg

0        24590.6965
1        19860.3340
2        25005.8240
3        25005.8240
4        18046.5560
            ...    
21462    23181.0850
21463    18046.5560
21464    25005.8240
21465    25005.8240
21466    19860.3340
Name: total_income, Length: 19347, dtype: float64

In [69]:
# Aplícalo a cada fila
data['total_income'].fillna(data_not_nan_avg, inplace = False)

0        40620.102
1        17932.802
2        23341.752
3        42820.568
4        25378.572
           ...    
21462    35966.698
21463    24959.969
21464    14347.610
21465    39054.888
21466    13127.587
Name: total_income, Length: 21467, dtype: float64

In [70]:
print(data['total_income'].dtype)

float64


In [71]:
# Comprueba si tenemos algún error
data['total_income'].isnull().sum()

2120

In [72]:
# Reemplazar los valores ausentes si hay algún error
#No hay ningun error


In [73]:
# Comprobar el número de entradas en las columnas
data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21467 entries, 0 to 21466
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21467 non-null  int64  
 1   days_employed     19347 non-null  float64
 2   dob_years         21467 non-null  int64  
 3   education         21467 non-null  object 
 4   education_id      21467 non-null  int64  
 5   family_status     21467 non-null  object 
 6   family_status_id  21467 non-null  int64  
 7   gender            21467 non-null  object 
 8   income_type       21467 non-null  object 
 9   debt              21467 non-null  int64  
 10  total_income      19347 non-null  float64
 11  purpose           21467 non-null  object 
 12  education_lower   21467 non-null  object 
 13  age_group         21467 non-null  object 
dtypes: float64(2), int64(5), object(7)
memory usage: 2.3+ MB


###  Restaurar valores en `days_employed`

In [74]:
# Distribución de las medianas de `days_employed` en función de los parámetros identificados
data['days_employed'].isna().sum()


2120

In [75]:
display(data.corr())

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
children,1.000000,-0.180776,-0.317403,-0.026659,-0.157880,0.024388,0.031455
days_employed,-0.180776,1.000000,0.570387,0.038294,-0.058406,-0.083757,-0.057476
dob_years,-0.317403,0.570387,1.000000,0.064417,-0.068531,-0.071371,-0.056063
education_id,-0.026659,0.038294,0.064417,1.000000,0.008162,0.052853,-0.178908
family_status_id,-0.157880,-0.058406,-0.068531,0.008162,1.000000,0.020647,-0.009151
debt,0.024388,-0.083757,-0.071371,0.052853,0.020647,1.000000,-0.012115
total_income,0.031455,-0.057476,-0.056063,-0.178908,-0.009151,-0.012115,1.000000


In [76]:
# Distribución de las medias de `days_employed` en función de los parámetros identificados
print(data_not_nan.groupby('dob_years')['days_employed'].median())

dob_years
19     724.492610
20     674.838979
21     618.733817
22     707.558942
23     690.204208
24     947.731043
25     919.199388
26    1083.658132
27    1166.212160
28    1141.705450
29    1315.453550
30    1420.586863
31    1302.524255
32    1446.622542
33    1426.415003
34    1615.910188
35    1613.494818
36    1799.520465
37    1816.713382
38    1817.194286
39    1872.259302
40    1728.936706
41    1864.657692
42    2253.981630
43    1855.522708
44    2084.330015
45    2250.937133
46    2100.473217
47    2203.078545
48    2429.674518
49    2560.317106
50    2626.986257
51    2846.080700
52    3395.639568
53    3650.007523
54    4026.541145
55    5537.000000
56    5537.000000
57    5537.000000
58    5537.000000
59    5537.000000
60    5537.000000
61    5537.000000
62    5537.000000
63    5537.000000
64    5537.000000
65    5537.000000
66    5537.000000
67    5537.000000
68    5537.000000
69    5537.000000
70    5537.000000
71    5537.000000
72    5537.000000
73    5537.000000


In [77]:
print(data_not_nan.groupby('income_type')['days_employed'].median())

income_type
business         1546.333214
civil servant    2689.368353
employee         1574.202821
retiree          5537.000000
Name: days_employed, dtype: float64


In [78]:
print(data_not_nan.groupby('gender')['days_employed'].median())

gender
F    2539.726923
M    1662.370103
Name: days_employed, dtype: float64


La mediana porque el valor no es muy significativa en days_employed

In [79]:
# Escribamos una función que calcule medias o medianas (dependiendo de tu decisión) según el parámetro identificado
data.groupby(['income_type','age_group','gender'])['days_employed'].median()

income_type    age_group     gender
business       adulto joven  F          691.238794
                             M          716.180179
               adulto mayor  F         1673.323646
                             M         1556.595932
               retirado      F         2598.981129
                             M         1913.690402
civil servant  adulto joven  F         1024.299332
                             M          842.682119
               adulto mayor  F         2746.045213
                             M         2802.226671
               retirado      F         3339.663548
                             M         3506.776656
employee       adulto joven  F          763.631105
                             M          772.799568
               adulto mayor  F         1798.951596
                             M         1425.031883
               retirado      F         3236.712208
                             M         3511.294028
retiree        adulto joven  F         5537.00

In [80]:
# Comprueba que la función funciona
data_not_nan_days_avg=data_not_nan.groupby(['income_type','age_group','gender'])['days_employed'].median()
data_not_nan_days_avg

income_type    age_group     gender
business       adulto joven  F          691.238794
                             M          716.180179
               adulto mayor  F         1673.323646
                             M         1556.595932
               retirado      F         2598.981129
                             M         1913.690402
civil servant  adulto joven  F         1024.299332
                             M          842.682119
               adulto mayor  F         2746.045213
                             M         2802.226671
               retirado      F         3339.663548
                             M         3506.776656
employee       adulto joven  F          763.631105
                             M          772.799568
               adulto mayor  F         1798.951596
                             M         1425.031883
               retirado      F         3236.712208
                             M         3511.294028
retiree        adulto joven  F         5537.00

In [81]:
# Aplicar la función al income_type

data['days_employed'].fillna('data_not_nan_days_avg', inplace = True)


In [82]:
# Comprueba si la función funcionó
data['days_employed'].isna().sum()

0

In [83]:
# Reemplazar valores ausentes

#La funcion funciono

In [84]:
# Comprueba las entradas en todas las columnas: asegúrate de que hayamos corregido todos los valores ausentes

data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21467 entries, 0 to 21466
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21467 non-null  int64  
 1   days_employed     21467 non-null  object 
 2   dob_years         21467 non-null  int64  
 3   education         21467 non-null  object 
 4   education_id      21467 non-null  int64  
 5   family_status     21467 non-null  object 
 6   family_status_id  21467 non-null  int64  
 7   gender            21467 non-null  object 
 8   income_type       21467 non-null  object 
 9   debt              21467 non-null  int64  
 10  total_income      19347 non-null  float64
 11  purpose           21467 non-null  object 
 12  education_lower   21467 non-null  object 
 13  age_group         21467 non-null  object 
dtypes: float64(1), int64(5), object(8)
memory usage: 2.3+ MB


## Ejercicio 4 Clasificación de datos



In [85]:
data['total_income'].apply(type).unique()

array([<class 'float'>], dtype=object)

In [86]:
filtro=data['total_income'].apply(lambda x: type(x) == 'str')

In [87]:
data.loc[filtro, 'total_income']

Series([], Name: total_income, dtype: float64)

In [88]:
# Muestra los valores de los datos seleccionados para la clasificación

data['purpose'].value_counts()

wedding ceremony                            793
having a wedding                            773
to have a wedding                           769
real estate transactions                    675
buy commercial real estate                  662
housing transactions                        652
buying property for renting out             651
transactions with commercial real estate    650
purchase of the house                       646
housing                                     646
purchase of the house for my family         638
construction of own property                634
property                                    633
transactions with my real estate            627
building a real estate                      625
buy real estate                             621
purchase of my own house                    620
building a property                         619
buy residential real estate                 606
housing renovation                          606
buying my own car                       

[Vamos a comprobar los valores únicos]

In [89]:
# Comprobar los valores únicos
data['purpose'].unique()

array(['purchase of the house', 'car purchase', 'supplementary education',
       'to have a wedding', 'housing transactions', 'education',
       'having a wedding', 'purchase of the house for my family',
       'buy real estate', 'buy commercial real estate',
       'buy residential real estate', 'construction of own property',
       'property', 'building a property', 'buying a second-hand car',
       'buying my own car', 'transactions with commercial real estate',
       'building a real estate', 'housing',
       'transactions with my real estate', 'cars', 'to become educated',
       'second-hand car purchase', 'getting an education', 'car',
       'wedding ceremony', 'to get a supplementary education',
       'purchase of my own house', 'real estate transactions',
       'getting higher education', 'to own a car', 'purchase of a car',
       'profile education', 'university education',
       'buying property for renting out', 'to buy a car',
       'housing renovation', 'going

Los agrupamientos que se van a  realizar de acuerdo al propisto del prestamo, son los siguientes;

1. Auto
2. Casa
3. boda
4. educacion 


In [90]:
# Escribamos una función para clasificar los datos en función de temas comunes

def purpose_groups (row):
    if 'car' in row['purpose']:
        return 'car'
    if 'hous' in row['purpose'] or 'prop' in row['purpose'] or 'real est' in row['purpose']:
        return 'house'
    if 'wedd' in row['purpose']:
        return 'wedding'
    if 'educ' in row['purpose'] or 'uni' in row['purpose']:
        return 'education'


In [91]:
# Crea una columna con las categorías y cuenta los valores en ellas

data['purpose_groups']= data.apply(purpose_groups ,axis=1)

In [92]:
# Revisar todos los datos numéricos en la columna seleccionada para la clasificación
data['purpose_groups'].value_counts()


house        10811
car           4307
education     4014
wedding       2335
Name: purpose_groups, dtype: int64

In [93]:
# Obtener estadísticas resumidas para la columna
data['purpose_groups'].describe()

count     21467
unique        4
top       house
freq      10811
Name: purpose_groups, dtype: object

La columna "total_income"  sera nuesta base para crear el rango numerico, la variable es la que más determina si la persona podra pagar o no, de acuerdo a sus ingresos. para eso se identificara primero el Quartil al que pertenece y luego se determinara las categorias

In [94]:
# Crear una función para clasificar en diferentes grupos numéricos basándose en rangos

print(data['total_income'].quantile([0.1,0.25,0.5,0.6, 0.85,0.99]))
print()

0.10    12177.58900
0.25    16494.01600
0.50    23203.78600
0.60    26346.11480
0.85    39508.25340
0.99    82767.73494
Name: total_income, dtype: float64



In [95]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21467 entries, 0 to 21466
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21467 non-null  int64  
 1   days_employed     21467 non-null  object 
 2   dob_years         21467 non-null  int64  
 3   education         21467 non-null  object 
 4   education_id      21467 non-null  int64  
 5   family_status     21467 non-null  object 
 6   family_status_id  21467 non-null  int64  
 7   gender            21467 non-null  object 
 8   income_type       21467 non-null  object 
 9   debt              21467 non-null  int64  
 10  total_income      19347 non-null  float64
 11  purpose           21467 non-null  object 
 12  education_lower   21467 non-null  object 
 13  age_group         21467 non-null  object 
 14  purpose_groups    21467 non-null  object 
dtypes: float64(1), int64(5), object(9)
memory usage: 2.5+ MB


In [96]:
# Crear una columna con categorías
def income_class(income):
    q_rico = data['total_income'].quantile(0.99)
    q_alto = data['total_income'].quantile(0.85)
    q_medio = data['total_income'].quantile(0.6)
    q_bajo = data['total_income'].quantile(0.25)
    if income < q_bajo:
        return 'pobre'
    if (income >= q_bajo) and (income < q_medio):
        return 'clase media baja'
    if (income >= q_medio) and (income < q_alto):
        return 'clase media'
    if (income >= q_alto) and (income < q_rico):
        return 'clase media alta'
    if (income >= q_rico):
        return 'rico'

In [ ]:
data['income_class'] = data['total_income'].apply(income_class)

In [ ]:
data['income_class'].value_counts()

## Ejercicio 5 Comprobación de las hipótesis


**¿Existe una correlación entre tener hijos y pagar a tiempo?**

In [ ]:
# Comprueba los datos sobre los hijos y los pagos puntuales


# Calcular la tasa de incumplimiento en función del número de hijos

pivot_children = data.pivot_table(index='children', columns='debt', values='days_employed', aggfunc='count')
pivot_children['defaulter_percent'] = pivot_children[1] / (pivot_children[1] + pivot_children[0])*100
pivot_children.sort_values(by='defaulter_percent')

**Conclusión**

De  acuerdo al analisis entre deuda y niños, los clientes que no tienen hijos son mas propensos a solicitar prestamos y son los menos porcentajes de morosidad tienen . y a medida que van teniando mas hijos, menos prestamos solicitan y la mororidad aumenta. no existe una diferencia grande entre incumplimiento de parte de las personas que tienen hijos.


**¿Existe una correlación entre la situación familiar y el pago a tiempo?**

In [ ]:
# Comprueba los datos del estado familiar y los pagos a tiempo

print(data.groupby('family_status')['debt'].sum())
print('*'*35)
print(data.groupby('family_status')['debt'].count())

# Calcular la tasa de incumplimiento basada en el estado familiar

pivot_status = data.pivot_table(index='family_status', columns='debt', values='days_employed', aggfunc='count')
pivot_status['defaulter_percent'] = pivot_status[1] / (pivot_status[1] + pivot_status[0])*100
pivot_status.sort_values(by='defaulter_percent')

**Conclusión**

los solteros tienen el porcentaje más alto de morosidad en comparación de los demas. y los viudos el porcentaje menor de morosidad, y a la vez, solocitan menos prestamos. 
Los casados tienen el numero mas alto de prestamos, pero comparandolo con los solteros, el porcentaje de morosidad es medio. 

**¿Existe una correlación entre el nivel de ingresos y el pago a tiempo?**

In [ ]:
# Comprueba los datos del nivel de ingresos y los pagos a tiempo
print(data.groupby('income_class')['debt'].sum())
print('*'*35)
print(data.groupby('income_class')['debt'].count())



# Calcular la tasa de incumplimiento basada en el nivel de ingresos

pivot_purpose = data.pivot_table(index='income_class', columns='debt', values='days_employed', aggfunc='count')
pivot_purpose['defaulter_percent'] = pivot_purpose[1] / (pivot_purpose[1] + pivot_purpose[0])*100
pivot_purpose.sort_values(by='defaulter_percent')


**Conclusión**

Los ricos son los que menos prestamos pide, y al pedirlo, tienen el menor porcentaje de morosidad en comparacion de los demas. los que piden mas pestamos es la clase media baja y es la que tiene el porcentaje de morosidad mas alto. pero comparando la diferencia entre todos los porcentajes, no existe  una diferencia muy alta entre todos. 

**¿Cómo afecta el propósito del crédito a la tasa de incumplimiento?**

In [ ]:
# Consulta los porcentajes de tasa de incumplimiento para cada propósito del crédito y analízalos
print(data.groupby('purpose_groups')['debt'].sum())
print('*'*35)
print(data.groupby('purpose_groups')['debt'].count())



# Calcular la tasa de incumplimiento basada en el nivel de ingresos

pivot_purpose = data.pivot_table(index='purpose_groups', columns='debt', values='days_employed', aggfunc='count')
pivot_purpose['defaulter_percent'] = pivot_purpose[1] / (pivot_purpose[1] + pivot_purpose[0])*100
pivot_purpose.sort_values(by='defaulter_percent')


**Conclusión**

Los creditos más solicitados son para comprar una casa, y son los que tienen el porcentaje de morosidad mas alta. luego le sigue un credito para boda, teniendo al igual que el anterior, un porcenjta de morosidad alto. los que son más propensos a pagar, son lo que solicitan un credito para comprar un auto.


# Conclusión general 

las etapas que se siguio en este analisis crediticio fue el siguiente;
1. exploracion de datos: se revisaron los datos para conocer a fondo el tipo de datos que nos han otorgado, con esto se identifico que habia un total 21525. y dos columnas presentaban valores ausentes; days_employed y total_income y el porcentaje  era del 10 %.


2. Transformacion de datos: se corrigieron las columnas que tenian errores de formato, carga o tipeo erroneo ( que puede ser un error de usario o sistema), la correcion de las columnas fueron las siguientes;
a. 'education': se corrigio el formato de las columnas, convirtiendolas todas en minusyla para que no las tomen como diferentes
b. 'children' : Los numeros negativos se convirtieron en  positivos y los muy altos se redujo 
c. 'dob_years': se presentaba el 0,4 % de valores que contenian cero, al ser poco significativo, se reemplazo por la mediana.
d. 'family_status': No se realizo ninguna modificacion porque los datos estaban correctos. y tampoco se hizo una agrupacion de datos, porque a veces es necesario conocer el detalle.
e.1.6 Gender: se identifico una variable "xna" que no tenia ninguna relacion con los datos, se reemplazo por el genero Femenino
f. ' income_type' : los tipos de trabajos que no tenian un numero significativo, se incorporaron en otros empleo, a los que más se adaptan 

Duplicados: se identificaron 54 datos duplicados, y se procedio a eliminarlos 

3. Correccion de los valores ausentes: las columnas que tenian valores ausentes eran days_employed y total_income y el porcentaje  correspondia al  10 %, al ser un valor poco significativo, se reemplazo por la mediana. en cado de total income, se uso la media de las variables que más influyen en el sueldo ('gender','education_id','income_type'). 
y con respecto a la variable " days_employee" se realizo lo mismo pero se calculo mediana de  (income_type','age_group','gender')

4. Categorizacion de datos: se categorizo el proposito del prestamo en la siguientes categorias [ car, weeding, house y education] y los ingresos de acuerdo al percentil que corresponde generando las siguientes categorias [pobre, clase media baja, clase media alta, rico]. esto permite el analisis del porcentaje de morosidad de acuerdo a los grupos creados

Conclusion de los creditos 

revisando las hipotesis y la correlacion entre las variables children, family status, grupo de ingresos y proposito del prestamo con el  porcentaje de morosidad. 

a. los clientes que no tienen hijos son mas propensos a solicitar prestamos y son los que tienen menos porcentajes de morosidad. y a medida que van teniando mas hijos, menos prestamos solicitan y la mororidad aumenta. no existe una diferencia grande entre incumplimiento de parte de las personas que tienen hijos.
B. los solteros tienen el porcentaje más alto de morosidad en comparación de los demas. y los viudos el porcentaje menor,y a la vez, solocitan menos prestamos. 
Los casados tienen el numero mas alto de prestamos, pero comparandolo con los solteros, el porcentaje de morosidad es medio.
C.Los ricos son los que menos prestamos pide, y al pedirlo, tienen el menor porcentaje de morosidad en comparacion de los demas. los que piden mas pestamos es la clase media baja y es la que tiene el porcentaje de morosidad mas alto. pero comparando la diferencia entre todos los porcentajes, no existe  una diferencia muy alta entre todos. 
D. Los creditos más solicitados son para comprar una casa, y son los que tienen el porcentaje de morosidad mas alta. luego le sigue un credito para boda, teniendo al igual que el anterior, un porcenjta de morosidad alto. los que son más propensos a pagar, son lo que solicitan un credito para comprar un auto.


En conslusion, a pesar de lo anterior,no existe un porcentaje tan diferencial entre cada categoria. pero si se quiere otorgar un prestamo, es recomedable otorgarlo un credito hipotecario a personas que no tienen hijos correspondiente a la clase media alta, ya que son los más propensos  a pagar  
